In [2]:
!pip install jiwer

In [2]:
!git clone git@github.com:mushrafi88/asr_bangla.git

Cloning into 'asr_bangla'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 223 (delta 9), reused 10 (delta 3), pack-reused 201
Receiving objects: 100% (223/223), 43.79 MiB | 237.00 KiB/s, done.
Resolving deltas: 100% (101/101), done.
Updating files: 100% (60/60), done.


In [1]:
# Getting Go SCTK CLI tool and giving it executable permissions.
!version=v0.3.0 && wget -q -O sctk https://github.com/shahruk10/go-sctk/releases/download/${version}/sctk && chmod +x ./sctk

# Printing usage docs.
!./sctk score --help

USAGE
  
sctk score \
  --ignore-first=true --delimiter="," --col-id=1 --col-trn=2 \
  --case-sensitive=false --normalize-unicode=true \
  --out=./wer --ref=truth.csv --hyp=output1.csv


FLAGS
  -case-sensitive=false  If true, scoring will be case sensitive.

  -cer=false  If true, will evaluate character error rate instead of word error rate.

  -col-id 0  The column index (zero based, positive only) containing <utteranceID>.

  -col-trn 1  The column index (zero based, positive only) containing <transcript>.

  -delimiter ,  The delimiter used in reference and hypotheses files. By default, the program expects
comma delimited files (.csv) The program needs at least two columns per row, containing
<utteranceID> and <transcript>. By default, the first and second columns are assumed
to contain <utteranceID> and <transcript> respectively. This can be changed by using
--id-col and --trn-col arguments.

  -encoding utf-8  What text encoding to use for interpreting text.

  -hyp ...  (Requir

## Transcribe validation set using pretrained model

- First let's generate some results using a pretrained off-the-shelf model

In [187]:
from typing import Dict, List, Tuple, Any, Union

import os

import numpy as np
import pandas as pd
import json

import transformers
from transformers import Wav2Vec2ForCTC, pipeline
from datasets import load_metric

from tqdm.auto import tqdm
from IPython.display import display, Audio, HTML

cer = load_metric("cer")
wer = load_metric("wer")
import warnings 
warnings.filterwarnings('ignore')

In [188]:
def space_remover(x):
    return " ".join(x.split())

In [189]:
def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’…,<>.‚/?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [190]:
df = pd.read_csv('asr_bangla/csv_files/df_nov_4gram.csv')
df.source.unique()

array(['train', 'validation', 'openslr'], dtype=object)

In [191]:
df.columns

Index(['path', 'id', 'source', 'audio', 'sentence', 'duration', 'snr',
       'wav2vec2', '2gram', '3gram', '4gram'],
      dtype='object')

In [192]:
df_train = df.drop(['id','duration','snr'],axis=1)

#df_train = df[(df.source == 'openslr') & (df.audio== 'noisy')]
df_train = df_train.drop(['source','audio'],axis=1)

df_train = df_train.dropna(how='any')

In [193]:
ref = df_train.drop(['wav2vec2','2gram','3gram','4gram'],axis=1)

pred = df_train.drop(['sentence','wav2vec2','2gram','3gram'],axis=1)

pred.rename(columns = {'4gram':'sentence'}, inplace = True)

In [194]:
pred

path  \
0      /kaggle/input/denoiser-for-benchmark/raw/train...   
1      /kaggle/input/denoiser-for-benchmark/raw/train...   
2      /kaggle/input/denoiser-for-benchmark/raw/train...   
3      /kaggle/input/denoiser-for-benchmark/raw/train...   
4      /kaggle/input/denoiser-for-benchmark/raw/train...   
...                                                  ...   
57361  /kaggle/input/denoiser-for-benchmark/denoised/...   
57362  /kaggle/input/denoiser-for-benchmark/denoised/...   
57363  /kaggle/input/denoiser-for-benchmark/denoised/...   
57364  /kaggle/input/denoiser-for-benchmark/denoised/...   
57365  /kaggle/input/denoiser-for-benchmark/denoised/...   

                                                sentence  
0      ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...  
1         তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক  
2      গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...  
3                   ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি  
4      অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...  
...                                                  ...  
57361                                    রাজনৈতিক সমাবেশ  
57362                                     এমন সংসদ সদস্য  
57363                                          ব্লেড গেল  
57364                                 তাদের ছেলে মেয়েদের  
57365                              জয়নাল আবেদিন এ তথ্যের  

[56897 rows x 2 columns]

In [195]:
ref

path  \
0      /kaggle/input/denoiser-for-benchmark/raw/train...   
1      /kaggle/input/denoiser-for-benchmark/raw/train...   
2      /kaggle/input/denoiser-for-benchmark/raw/train...   
3      /kaggle/input/denoiser-for-benchmark/raw/train...   
4      /kaggle/input/denoiser-for-benchmark/raw/train...   
...                                                  ...   
57361  /kaggle/input/denoiser-for-benchmark/denoised/...   
57362  /kaggle/input/denoiser-for-benchmark/denoised/...   
57363  /kaggle/input/denoiser-for-benchmark/denoised/...   
57364  /kaggle/input/denoiser-for-benchmark/denoised/...   
57365  /kaggle/input/denoiser-for-benchmark/denoised/...   

                                                sentence  
0      ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...  
1         তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক  
2      গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...  
3                 ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি  
4      অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...  
...                                                  ...  
57361                                    রাজনৈতিক সমাবেশ  
57362                                     এমন সংসদ সদস্য  
57363                                          ব্লেড গেল  
57364                                  তাদের ছেলেমেয়েদের  
57365                              জয়নাল আবেদীন এ তথ্যের  

[56897 rows x 2 columns]

In [196]:
ref.to_csv('valid_ref.csv',index=False)
pred.to_csv('valid_pred.csv',index=False)

In [197]:
# Running sctk on model output for the validation set.
# 
# Since the first row in the file contains headers, we set --ignore-first=true
#
# The file ID is the first column (index = 0) and the transcripts
# are in the second column (index = 1); so setting --col-id and
# --col-trn to 0 and 1 respectively.
#
# By setting --cer=false, we are evaluating at the word level.
!./sctk score \
  --ignore-first=true \
  --delimiter="," \
  --col-id=0 \
  --col-trn=1 \
  --cer=false \
  --out=report \
  --ref=valid_ref.csv \
  --hyp=valid_pred.csv

- SCTK generates a bunch of files. Let's take a look at some.

- First, the `*.dtl` file contains a detailed breakdown of the different errors; there are three sections: `Confusion Pairs` or `Substitutions`, `Deletions` and `Insertions`.

In [198]:
#display(HTML(open("./report/hyp1.trn.dtl", 'r').read().replace("\n", "<br>")))

- Next, let's take a look at the `*.pra` file. This contains alignments between the reference and model output. This is provided in a variety of formats. HTML is probably the easiest to view in a notebook.

 - Sidenote, these alignments can also be used to combine the outputs from different ASR models (See [ROVER](https://ieeexplore.ieee.org/document/659110) algorithm).


In [199]:
#display(HTML(open("./report/hyp1.trn.pra.html", 'r').read()))

In [200]:
label = '4gram_label'
wer_percentage = '4gram_wer'

In [201]:
import json
with open('./report/hyp1.trn.pra.json') as json_file:
    data = json.load(json_file)

In [202]:
speakers = list(data['speakers'].keys())[0]

In [203]:
df_errors = pd.DataFrame()

l=list(data['speakers'][speakers].keys())

for i in tqdm(range(len(l))):
    temp=data['speakers'][speakers][l[i]]
    sen_id=temp['sentence_id']
    sen_id=sen_id[1:-1]
    words=temp['words']

    actual_sen = " ".join([x['ref'] for x in words])
    pred_sen = " ".join([x['hyp'] for x in words])
    labels = [x['eval_label'] for x in words]
    #list1=['S','D','I','C']
    #if any(x in list1 for x in labels):
    dict_ = {'path': sen_id,label:labels}
    df_errors = df_errors.append(dict_, ignore_index = True)

  0%|          | 0/49152 [00:00<?, ?it/s]

In [204]:
def wer(label):
    s=label.count('S')
    d=label.count('D')
    i=label.count('I')
    return int((s+d+i)/len(label)*100)

df_errors[wer_percentage] = df_errors[label].apply(lambda x:wer(x))
#df_errors.path=df_errors.path.str.extract('(\d+)')
df_errors=df_errors.sort_values(by=wer_percentage, ascending=False)

In [205]:
df_errors

path  \
45759  /kaggle/input/denoiser-for-benchmark/raw/train...   
32914  /kaggle/input/denoiser-for-benchmark/raw/slr53...   
47667  /kaggle/input/denoiser-for-benchmark/raw/train...   
43493  /kaggle/input/denoiser-for-benchmark/raw/train...   
34694  /kaggle/input/denoiser-for-benchmark/raw/slr53...   
...                                                  ...   
35565  /kaggle/input/denoiser-for-benchmark/raw/slr53...   
35564  /kaggle/input/denoiser-for-benchmark/raw/slr53...   
35563  /kaggle/input/denoiser-for-benchmark/raw/slr53...   
18769  /kaggle/input/denoiser-for-benchmark/denoised/...   
0      /kaggle/input/denoiser-for-benchmark/denoised/...   

                                         4gram_label  4gram_wer  
45759                             [I, S, S, S, S, S]        100  
32914                                         [D, S]        100  
47667        [D, D, D, D, D, D, D, D, D, D, D, D, S]        100  
43493                                      [S, S, S]        100  
34694                                   [I, I, S, S]        100  
...                                              ...        ...  
35565                                         [C, C]          0  
35564                                         [C, C]          0  
35563                                         [C, C]          0  
18769  [C, C, C, C, C, C, C, C, C, C, C, C, C, C, C]          0  
0                                    [C, C, C, C, C]          0  

[49152 rows x 3 columns]

In [206]:
df_errors[df_errors[wer_percentage] == 0]

path  \
8566   /kaggle/input/denoiser-for-benchmark/denoised/...   
8541   /kaggle/input/denoiser-for-benchmark/denoised/...   
8500   /kaggle/input/denoiser-for-benchmark/denoised/...   
8525   /kaggle/input/denoiser-for-benchmark/denoised/...   
8545   /kaggle/input/denoiser-for-benchmark/denoised/...   
...                                                  ...   
35565  /kaggle/input/denoiser-for-benchmark/raw/slr53...   
35564  /kaggle/input/denoiser-for-benchmark/raw/slr53...   
35563  /kaggle/input/denoiser-for-benchmark/raw/slr53...   
18769  /kaggle/input/denoiser-for-benchmark/denoised/...   
0      /kaggle/input/denoiser-for-benchmark/denoised/...   

                                         4gram_label  4gram_wer  
8566                                       [C, C, C]          0  
8541                                    [C, C, C, C]          0  
8500                                          [C, C]          0  
8525                                 [C, C, C, C, C]          0  
8545                                    [C, C, C, C]          0  
...                                              ...        ...  
35565                                         [C, C]          0  
35564                                         [C, C]          0  
35563                                         [C, C]          0  
18769  [C, C, C, C, C, C, C, C, C, C, C, C, C, C, C]          0  
0                                    [C, C, C, C, C]          0  

[18938 rows x 3 columns]

In [207]:
#df_errors.rename(columns = {'wav2vec2':'wav2vec2_label'}, inplace = True)

In [208]:
df = pd.read_csv('final.csv')
df=df.merge(df_errors,on='path',how='inner')

In [209]:
df

path          id   source  \
0      /kaggle/input/denoiser-for-benchmark/raw/train...    30991599    train   
1      /kaggle/input/denoiser-for-benchmark/raw/train...    30991687    train   
2      /kaggle/input/denoiser-for-benchmark/raw/train...    30991851    train   
3      /kaggle/input/denoiser-for-benchmark/raw/train...    30992090    train   
4      /kaggle/input/denoiser-for-benchmark/raw/train...    30992661    train   
...                                                  ...         ...      ...   
49147  /kaggle/input/denoiser-for-benchmark/denoised/...  056711c865  openslr   
49148  /kaggle/input/denoiser-for-benchmark/denoised/...  a3230c3684  openslr   
49149  /kaggle/input/denoiser-for-benchmark/denoised/...  7f87c122cb  openslr   
49150  /kaggle/input/denoiser-for-benchmark/denoised/...  a6463ecbb0  openslr   
49151  /kaggle/input/denoiser-for-benchmark/denoised/...  c6ad019cee  openslr   

          audio                                           sentence  duration  \
0         noisy  ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...     5.544   
1         noisy     তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক     4.032   
2         noisy  গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...     4.320   
3         noisy             ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি     4.284   
4         noisy  অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...     5.760   
...         ...                                                ...       ...   
49147  denoised                                    রাজনৈতিক সমাবেশ     3.168   
49148  denoised                                     এমন সংসদ সদস্য     3.348   
49149  denoised                                          ব্লেড গেল     2.736   
49150  denoised                                  তাদের ছেলেমেয়েদের     4.536   
49151  denoised                              জয়নাল আবেদীন এ তথ্যের     3.060   

             snr                                           wav2vec2  \
0     -44.277168  ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...   
1     -46.880779     তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক   
2     -56.581566  গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...   
3     -29.025093             ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি   
4     -32.322632  অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...   
...          ...                                                ...   
49147 -37.201962                                    রাজনৈতিক সমাপেশ   
49148 -36.750912                                     এমন সংসদ সদস্য   
49149 -26.636936                                         ল্লেড গেলো   
49150 -18.375225                                 তাদের ছেলে মেয়েদের   
49151 -35.931503                             জনাল আবেদদিন এ ততথ্যের   

                                                   2gram  \
0      ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...   
1         তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক   
2      গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...   
3                   ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি   
4      অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...   
...                                                  ...   
49147                                    রাজনৈতিক সমাবেশ   
49148                                     এমন সংসদ সদস্য   
49149                                          ব্লেড গেল   
49150                                 তাদের ছেলে মেয়েদের   
49151                              জয়নাল আবেদিন এ তথ্যের   

                                                   3gram  \
0      ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...   
1         তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক   
2      গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...   
3                   ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি   
4      অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...   
...                                                  ...   
49147                                    রাজনৈতিক সমাবেশ   
49148                                     এমন সংসদ সদস্য   
4914

In [210]:
df.to_csv('final.csv',index=False)